In [1]:
from p2pfl.node import Node
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(host="localhost")
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía

        
test_convergence((2,2))

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpfgo0gw8l
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpfgo0gw8l/_remote_module_non_sriptable.py
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en localhost 49820
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en localhost 49821
INFO:root:Conexión aceptada con ('localhost', 49821)
INFO:root:Broadcasting start learning...
INFO:root:Training...
INFO:root:Training...
INFO:root:Model added (1/2)
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (2/2)
INFO:root:Agregating models.
[54000, 54000]
INFO:root:Round 1 of 2 finished. (('localhost', 49821))
INFO:root:Model added (2/2)
INFO:root:Agregating models.
[54000, 54000]
INFO:root:Round 1 of 2 finished. (('localhost', 49820))
INFO:root:Train

In [ ]:
from p2pfl.const import HEARTBEAT_FREC, TIEMOUT
from p2pfl.node import Node
import time

n1 = Node("localhost")
n2 = Node("localhost")
n1.start()
n2.start()

def test_interrupt_train():
    n1 = Node("localhost")
    n1.start()
    n1.set_start_learning(99999,99999)

    time.sleep(1) #Esperar por la asincronía

    n1.set_stop_learning()
    n1.stop()
    
test_interrupt_train()

In [2]:
from p2pfl.learning.agregators.fedavg import FedAvg
from p2pfl.learning.pytorch.models.mlp import MLP
from p2pfl.learning.pytorch.learners.lightninglearner import LightningLearner
from collections import OrderedDict
import torch
def test_avg_complex():
    agregator = FedAvg(None)
    nl1 = LightningLearner(MLP(), None)
    params = nl1.get_parameters()
    params1 = nl1.get_parameters()
    params2 = nl1.get_parameters()

    result = agregator.agregate([(params,1)])

    # Check Results
    for layer in params:
        assert torch.eq(params[layer], result[layer]).all()

    for layer in params2:
        params1[layer] = params1[layer]+1
        params2[layer] = params2[layer]-1
    
    result = FedAvg.agregate([(params1,1), (params2,1)])

    # Check Results -> Careful with rounding errors
    for layer in params:
        a = torch.trunc(params[layer]*10)
        b = torch.trunc(result[layer]*10)
        assert torch.eq(a, b).all()

test_avg_complex()

TypeError: agregate() missing 1 required positional argument: 'models'

In [7]:
sum = [torch.zeros_like(sum[layer]) for layer in sum]
print(sum)

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [

In [11]:
from p2pfl.learning.agregators.fedavg import FedAvg
from p2pfl.learning.pytorch.models.mlp import MLP
from p2pfl.learning.pytorch.learners.lightninglearner import LightningLearner
from collections import OrderedDict
import torch



def test_avg_simple():

    a = OrderedDict([('a', torch.tensor(-1)), ('b', torch.tensor(-1))])
    b = OrderedDict([('a', torch.tensor(0)), ('b', torch.tensor(0))])
    c = OrderedDict([('a', torch.tensor(1)), ('b', torch.tensor(1))])

    result = FedAvg.agregate([(a,1),(b,1),(c,1)])

    for layer in b:
        assert result[layer] == b[layer]
        
test_avg_simple()

In [1]:
from p2pfl.learning.agregators.fedavg import FedAvg
from p2pfl.learning.pytorch.models.mlp import MLP
from p2pfl.learning.pytorch.learners.lightninglearner import LightningLearner
from collections import OrderedDict
import torch


def test_avg_complex():
    nl1 = LightningLearner(MLP(), None)
    params = nl1.get_parameters()
    params1 = nl1.get_parameters()
    params2 = nl1.get_parameters()

    result = FedAvg.agregate([(params,1)])

    # Check Results
    for layer in params:
        assert torch.eq(params[layer], result[layer]).all()

    for layer in params2:
        params1[layer] = params1[layer]+1
        params2[layer] = params2[layer]-1
    
    result = FedAvg.agregate([(params1,1), (params2,1)])

    # Check Results
    for layer in params:
        a = torch.trunc(params[layer]*10)
        b = torch.trunc(result[layer]*10)
        assert torch.eq(a, b).all()
        
while True:
    test_avg_complex()

KeyboardInterrupt: 

SyntaxError: invalid syntax (2964970389.py, line 1)

In [ ]:
from p2pfl.node import Node


n1 = Node("localhost")
n1.start()

n1.learner

In [2]:
[True].all()

AttributeError: 'list' object has no attribute 'all'

In [3]:
def myFun(a,*argv): 
    for arg in argv: 
        print (arg)

[myFun(cmd,args) for cmd,args in [("a",["b","c"])]]

['b', 'c']


[None]